## Kim Vo
### Part 1 (non-extra credit):

In [1]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

In [3]:
# Add field "isUrban" for data in 2017
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp",  "isUrban", "TEXT")

In [36]:
# Update "isUrban" field 
fields = ['isUrban','ALANDMI','POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
for row in curU:
    if (row[1]*1000) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
    else:
        row[0] = "N"
        curU.updateRow(row)
del curU

In [38]:
# Print out percentage of the population of the state is urbanized in 2017
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp","POP2017",
                             where_clause= '"ALANDMI" > 0 and "POP2017" > 0')
                            # use where_clause to select eliminate rows without data
total = 0.0
for row in curS:
    total += 1

countUrban = 0.0
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp","isUrban")
for row in curS:
    if row[0] == "Y":
        countUrban += 1

print countUrban/total*100
del curS

71.821233739


In [39]:
# Print out percentage of the land area of the state is urbanized in 2017
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp",["ALANDMI","isUrban"])
urbanLand = 0.0
totalLand = 0.0
for row in curS:
    totalLand += row[0]
    if row[1] == "Y":
        urbanLand += row[0]
        
print urbanLand/totalLand*100
del curS

2.2669959616


In [40]:
# Print out number of block groups urbanized and deurbanized from 2007 to 2017\

# Add new fields "isUrban07" to show urban/rural status for 2007
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp", "isUrban07", "TEXT")
fields = ['isUrban07','ALANDMI','POP2007']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
for row in curU:
    if (row[1]*1000) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
    else:
        row[0] = "N"
        curU.updateRow(row)

# Add new field "urbanize" to show changes from 2007 - 2017
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp", "urbanize", "TEXT")
fields = ['isUrban', 'isUrban07','urbanize','ALANDMI','POP2007', 'POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
for row in curU:
    if row[3] > 0 and row[4] > 0 and row[5] > 0:
        if row[0] == row[1]:
            row[2] = "no change in category"
            curU.updateRow(row)
        elif row[0] == "Y" and row[1] == "N":
            row[2]= "urbanized"
            curU.updateRow(row)
        else:
            row[2] = "deurbanized"
            curU.updateRow(row)

# Print out result
cursor = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp", ["urbanize"])
urbanized = 0
deurbanized = 0
for row in cursor:
    if(row[0] == "urbanized"):
        urbanized += 1
    elif(row[0] == "deurbanized"):
        deurbanized += 1
print "Block groups urbanized: " + str(urbanized)
print "Block groups deurbanized: " + str(deurbanized)

del curU
del cursor

Block groups urbanized: 72
Block groups deurbanized: 3


### 20% Challenge:
#### Find percentages of numbers of urban block groups change:

In [19]:
# Find the percentage of blocks urbanized in 2007 and print out the result
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp","POP2007",
                             where_clause= '"ALANDMI" > 0 and "POP2007" > 0')
                            # use where_clause to select eliminate rows without data
total = 0.0
for row in curS:
    total += 1

countUrban = 0.0
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp","isUrban07")
for row in curS:
    if row[0] == "Y":
        countUrban += 1

result = round(countUrban/total*100, 2) # reduce result to 2 decimals
print str(result) + "% of blocks were urbanized in 2007"

# Use result from previous calculation for urban/rural status in 2017 to find overall change
currentUrbanPct = 71.82
print str(currentUrbanPct) + "% of blocks were urbanized in 2017"
print str(currentUrbanPct - result) + "% is the change in WA urbanization between 2007 - 2017"

del curS

70.33% of blocks were urbanized in 2007
71.82% of blocks were urbanized in 2017
1.49% is the change in WA urbanization between 2007 - 2017


#### Find the % change for urban-area populations of the state: 

In [44]:
# Find the % of urban population in 2007 and print out result
totalPop07 = 0
urbanPop07 = 0
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp",["POP2007", "isUrban07"])
for row in curS:
    totalPop07 += row[0]
    if row[1] == "Y":
        urbanPop07 += row[0]
result07 = round(urbanPop07/totalPop07*100, 2)
print str(result07) + "% of the state population were urbanized in 2007"

# Find the % of urban population in 2017 and print out result
totalPop17 = 0
urbanPop17 = 0
curS = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp",["POP2017", "isUrban"])
for row in curS:
    totalPop17 += row[0]
    if row[1] == "Y":
        urbanPop17 += row[0]
result17 = round(urbanPop17/totalPop17*100, 2)
print str(result17) + "% of the state population were urbanized in 2007"

# Print out overall change
print str(result17 - result07) + "% of the state's urban-area population change from 2007-2017"

del curS

70.76% of the state population were urbanized in 2007
72.98% of the state population were urbanized in 2007
2.22% of the state's urban-area population change from 2007-2017


#### Run analysis and compare results for 20% decreases and increases in the thredshold between urban and rural:

Since I used 1000 persons/square-mile for previous analysis, I will recalculate numbers of urbanized and deurbanized blocks with the thredshold for urbanized areas being >= 800 persons/square-mile, and >= 1200 persons/square-mile.

In [52]:
# Find numbers of urbanized & deurbanized blocks from 2007-2017 with at least 800 persons/square-mile for urban area

# Print current result for urbanized & deurbanized blocks from previous calculation for comparison purpose
print "Urbanization with >= 1000 persons/square-mile (from previous calculation):"
print "\t Block groups urbanized: 72"
print "\t Block groups deurbanized: 3 \n"

print "Urbanization with >= 800 persons/square-mile:"
arcpy.env.overwriteOutput = True
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp",  "isUrb07Low", "TEXT")
# Update "isUrb07Low" field 
fields = ['isUrb07Low','ALANDMI','POP2007']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
countUrb07Low = 0
for row in curU:
    if (row[1]*800) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
        countUrb07Low += 1
    else:
        row[0] = "N"
        curU.updateRow(row)

arcpy.AddField_management("U:\GEOG458\saep_bg10.shp",  "isUrb17Low", "TEXT")
# Update "isUrb17Low" field for 2017
fields = ['isUrb17Low','ALANDMI','POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
countUrb17Low = 0
for row in curU:
    if (row[1]*800) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
        countUrb17Low += 1
    else:
        row[0] = "N"
        curU.updateRow(row)

# Add new field "changeLow" to show changes from 2007 - 2017
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp", "changeLow", "TEXT")
fields = ['isUrb17Low', 'isUrb07Low','changeLow','ALANDMI','POP2007', 'POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
for row in curU:
    if row[3] > 0 and row[4] > 0 and row[5] > 0:
        if row[0] == row[1]:
            row[2] = "no change in category"
            curU.updateRow(row)
        elif row[0] == "Y" and row[1] == "N":
            row[2]= "urbanized"
            curU.updateRow(row)
        else:
            row[2] = "deurbanized"
            curU.updateRow(row)

# Print out result
cursor = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp", ["changeLow"])
urbanized = 0
deurbanized = 0
for row in cursor:
    if(row[0] == "urbanized"):
        urbanized += 1
    elif(row[0] == "deurbanized"):
        deurbanized += 1
print "\t Block groups urbanized: " + str(urbanized)
print "\t Block groups deurbanized: " + str(deurbanized) + "\n"


print "Urbanization with >= 1200 persons/square-mile:"
arcpy.env.overwriteOutput = True
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp",  "isUrb07H", "TEXT")
# Update "isUrb07H" field 
fields = ['isUrb07H','ALANDMI','POP2007']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
countUrb07H = 0
for row in curU:
    if (row[1]*1200) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
        countUrb07H += 1
    else:
        row[0] = "N"
        curU.updateRow(row)

arcpy.AddField_management("U:\GEOG458\saep_bg10.shp",  "isUrb17H", "TEXT")
# Update "isUrb17H" field for 2017
fields = ['isUrb17H','ALANDMI','POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
countUrb17H = 0
for row in curU:
    if (row[1]*1200) <= row[2] and row[1] > 0 and row[2] > 0:
        row[0] = "Y"
        curU.updateRow(row)
        countUrb17H += 1
    else:
        row[0] = "N"
        curU.updateRow(row)

# Add new field "changeH" to show changes from 2007 - 2017
arcpy.AddField_management("U:\GEOG458\saep_bg10.shp", "changeH", "TEXT")
fields = ['isUrb17H', 'isUrb07H','changeH','ALANDMI','POP2007', 'POP2017']
curU = arcpy.da.UpdateCursor("U:\GEOG458\saep_bg10.shp",fields)
for row in curU:
    if row[3] > 0 and row[4] > 0 and row[5] > 0:
        if row[0] == row[1]:
            row[2] = "no change in category"
            curU.updateRow(row)
        elif row[0] == "Y" and row[1] == "N":
            row[2]= "urbanized"
            curU.updateRow(row)
        else:
            row[2] = "deurbanized"
            curU.updateRow(row)

# Print out result
cursor = arcpy.da.SearchCursor("U:\GEOG458\saep_bg10.shp", ["changeH"])
urbanized = 0
deurbanized = 0
for row in cursor:
    if(row[0] == "urbanized"):
        urbanized += 1
    elif(row[0] == "deurbanized"):
        deurbanized += 1
print "\t Block groups urbanized: " + str(urbanized)
print "\t Block groups deurbanized: " + str(deurbanized)

Urbanization with >= 1000 persons/square-mile (from previous calculation):
	 Block groups urbanized: 72
	 Block groups deurbanized: 3 

Urbanization with >= 800 persons/square-mile:
	 Block groups urbanized: 61
	 Block groups deurbanized: 1

Urbanization with >= 1200 persons/square-mile:
	 Block groups urbanized: 65
	 Block groups deurbanized: 3


#### This code block is very long, but I put them all in one block because I want the results to be printed out next to each other for comparison purpose.